<a href="https://colab.research.google.com/github/igorgcgv/igorgcgv/blob/main/Projeto_DBContoso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Configurando o Ambiente de Trabalho

In [ ]:

!pip install  -q ipython-sql

import pandas as pd
import sqlite3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.0 MB/s eta 0:00:00


In [ ]:
#Criando uma conexão com o banco de dados
cnn = sqlite3.connect ('contoso.db')
cursor = cnn.cursor ()

In [ ]:
#Fazendo a carga das tabelas (datasets)
factsales = pd.read_csv ('/content/factsales.csv', sep =';', error_bad_lines=False)
dimemployee = pd.read_csv ('/content/dimemployee.csv', sep =';', error_bad_lines=False)
dimproduct = pd.read_csv ('/content/dimproduct.csv', sep =';', error_bad_lines=False)
dimstore = pd.read_csv ('/content/dimstore.csv', sep =';', error_bad_lines=False)
dimcustomer = pd.read_csv ('/content/dimcustomer.csv', sep =';', error_bad_lines=False)
dimchannel = pd.read_csv ('/content/dimchannel.csv', sep =';', error_bad_lines=False)
dimdate = pd.read_csv ('/content/dimdate.csv', sep =';', error_bad_lines=False)
dimproductcategory = pd.read_csv ('/content/dimproductcategory.csv', sep =';', error_bad_lines=False)
dimproductsubcategory = pd.read_csv ('/content/dimproductsubcategory.csv', sep =';', error_bad_lines=False)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#Transformando o DataFrame em SQL para armazenar no banco de dados criado acima!
factsales.to_sql('factsales', cnn)
dimemployee.to_sql('dimemployee', cnn)
dimproduct.to_sql('dimproduct', cnn)
dimstore.to_sql('dimstore', cnn)
dimcustomer.to_sql('dimcustomer', cnn)
dimchannel.to_sql('dimchannel', cnn)
dimdate.to_sql('dimdate', cnn)
dimproductcategory.to_sql('dimproductcategory', cnn)
dimproductsubcategory.to_sql('dimproductsubcategory', cnn)

###Analisando Dados do BD Contoso com SQL (Group By, Order By)


 /*O gerente comercial pediu a você uma análise da Quantidade Vendida e
Quantidade Devolvida para o canal de venda mais importante da
empresa: Store.
Utilize uma função SQL para fazer essas consultas no seu banco de dados.
Obs: Faça essa análise considerando a tabela FactSales.*/


In [ ]:
consulta_1 = """
SELECT
	F.StoreKey AS 'ID Loja',
	D.StoreName AS 'Loja',
	SUM(SalesQuantity) AS 'Quantidade Vendida'
	
FROM FactSales F
INNER JOIN DimStore D
	ON F.StoreKey = D.StoreKey
GROUP BY F.StoreKey,D.StoreName
ORDER BY SUM(SalesQuantity) DESC


"""
#Com essa consulta temos todas as lojas ordenadas pela a 'Quantidade Vendida' do maio para o menor.
pd.read_sql(consulta_1, cnn)

,ID Loja,Loja,Quantidade Vendida
0,200,Contoso Catalog Store,16768
1,307,Contoso Asia Online Store,14936
2,199,Contoso North America Online Store,14315
3,306,Contoso Europe Online Store,9714
4,308,Contoso North America Reseller,8297
...,...,...,...
300,90,Contoso Dallas Store,54
301,65,Contoso Racine No.2 Store,39
302,228,Contoso Berlin Store,26
303,258,Contoso Thimphu No.1 Store,20


/* Faça um agrupamento mostrando a quantidade total vendida
(SalesQuantity) e quantidade total devolvida (Return Quantity) de
acordo com o ID das lojas (StoreKey).*/

In [ ]:
consulta_2 = """
SELECT
	F.StoreKey AS 'ID Loja',
	D.StoreName AS 'Loja',
	SUM(SalesQuantity) AS 'Quantidade Vendida',
	SUM(ReturnQuantity) AS 'Total Devolvido'
FROM FactSales F
INNER JOIN DimStore D
	ON F.StoreKey = D.StoreKey
GROUP BY F.StoreKey,D.StoreName
ORDER BY SUM(SalesQuantity) DESC


"""
pd.read_sql(consulta_2, cnn)

##Com essa consulta temos todas as lojas ordenadas pela a 'Quantidade Vendida' do maio para o menor e também a informação do Total Devolvido.

,ID Loja,Loja,Quantidade Vendida,Total Devolvido
0,200,Contoso Catalog Store,16768,131
1,307,Contoso Asia Online Store,14936,98
2,199,Contoso North America Online Store,14315,140
3,306,Contoso Europe Online Store,9714,101
4,308,Contoso North America Reseller,8297,81
...,...,...,...,...
300,90,Contoso Dallas Store,54,1
301,65,Contoso Racine No.2 Store,39,1
302,228,Contoso Berlin Store,26,0
303,258,Contoso Thimphu No.1 Store,20,0


/*Faça um resumo do valor total vendido (SalesAmount) para cada
canal de venda, mas apenas para o ano de 2007.*/

In [ ]:
consulta_3 = """
SELECT 
 
	F.channelKey AS 'ID',
	D.channelName AS 'Canal de Venda',
	ROUND(SUM(SalesAmount),2) AS 'Total Vendido ($)'
FROM FactSales F
INNER JOIN DimChannel D
	ON F.channelKey = D.channelKey
WHERE DateKey >= '01/01/2007' AND DateKey <= '31/12/2007'
GROUP BY F.channelKey, D.channelName
"""
pd.read_sql(consulta_3, cnn)

#Com essa consulta temos um resumo sintetico do total vendido em Dolar ($) no ano de 2017, segmentada por Tipo de Canal de Venda.

,ID,Canal de Venda,Total Vendido ($)
0,1,Store,19741430.0
1,2,Online,8127779.0
2,3,Catalog,3524288.0
3,4,Reseller,4981029.0



/*Você precisa fazer uma análise de vendas por produtos. O objetivo final é
descobrir o valor total vendido (SalesAmount) por produto (ProductKey).
a. A tabela final deverá estar ordenada de acordo com a quantidade
vendida e, além disso, mostrar apenas os produtos que tiveram um
resultado final de vendas maior do que $50.000*/

In [ ]:
consulta_4 = """
SELECT 
 
	F.ProductKey AS 'ID',
	D.ProductName AS 'Produto',
	ROUND(SUM(SalesAmount),2) AS 'Total Vendido ($)'
FROM FactSales F
INNER JOIN DimProduct D
	ON F.ProductKey = D.ProductKey
GROUP BY F.ProductKey,D.ProductName
HAVING ROUND(SUM(SalesAmount),2) > 50000
ORDER BY ROUND(SUM(SalesAmount),2) DESC 

"""
pd.read_sql(consulta_4, cnn)
#Com essa consulta temos um resumo sintetico do total vendido em Dolar ($), segmentado por produtos que estão com o total vendido acima dos 50.000,00 ($)

,ID,Produto,Total Vendido ($)
0,539,Proseware Projector 1080p LCD86 Black,202097.0
1,551,Proseware Projector 1080p LCD86 White,189221.0
2,1864,NT Washer & Dryer 24in M2400 Green,186434.0
3,540,Proseware Projector 1080p DLP86 Black,179927.0
4,575,Contoso Projector 1080p X980 Black,172584.0
...,...,...,...
145,1108,Fabrikam SLR Camera X146 Black,50859.0
146,371,Adventure Works Laptop16 M1601 White,50795.0
147,1409,The Phone Company Touch Screen Phone 1600 TFT-...,50446.0
148,1876,Contoso Washer & Dryer 15.5in E155 Silver,50399.0


/*Faça uma adaptação no exercício anterior e mostre os Top 10
produtos com mais vendas. Desconsidere o filtro de $5.000.000
aplicado.*/

In [ ]:
consulta_5 = """
SELECT 
 
	F.ProductKey AS 'ID',
	D.ProductName AS 'Produto',
	ROUND(SUM(SalesAmount),2) AS 'Total Vendido ($)'
FROM FactSales F
INNER JOIN DimProduct D
	ON F.ProductKey = D.ProductKey
GROUP BY F.ProductKey,D.ProductName
HAVING ROUND(SUM(SalesAmount),2) > 50000
ORDER BY ROUND(SUM(SalesAmount),2) DESC
LIMIT 10
"""
pd.read_sql(consulta_5, cnn)
#Com essa consulta temos um resumo sintetico do total vendido em Dolar ($), segmentado por produtos que estão com o total vendido acima dos 50.000,00 ($) 
#com os Top 10 Produtos.

,ID,Produto,Total Vendido ($)
0,539,Proseware Projector 1080p LCD86 Black,202097.0
1,551,Proseware Projector 1080p LCD86 White,189221.0
2,1864,NT Washer & Dryer 24in M2400 Green,186434.0
3,540,Proseware Projector 1080p DLP86 Black,179927.0
4,575,Contoso Projector 1080p X980 Black,172584.0
5,1957,Litware Refrigerator 24.7CuFt X980 Green,163835.0
6,1852,NT Washer & Dryer 27in L2700 Silver,161797.0
7,2090,Contoso Water Heater 7.2GPM X1800 Blue,159447.0
8,587,Contoso Projector 1080p X980 White,157437.0
9,2085,Contoso Water Heater 7.2GPM X1800 White,156423.0



/*Uma nova ação no setor de Marketing precisará avaliar a média salarial
de todos os clientes da empresa, mas apenas de ocupação Professional.
Utilize um comando SQL para atingir esse resultado.*/


In [ ]:
consulta_6 = """
SELECT
	Occupation AS 'Ocupação',
	ROUND(AVG(YearlyIncome),2) AS 'Media Salarial'
FROM DimCustomer
WHERE Occupation = 'Professional'
GROUP BY Occupation
"""
pd.read_sql(consulta_6, cnn)
#Com essa consulta temos que pra ocupação 'Professional' a média salarial é de   $ 74,184.78 por ano.

,Ocupação,Media Salarial
0,Professional,74184.78


/*Você precisará fazer uma análise da quantidade de funcionários das lojas
registradas na empresa. O seu gerente te pediu os seguintes números e
informações:
a. Quantos funcionários tem a loja com mais funcionários?*/

In [ ]:
consulta_7 = """
SELECT
	StoreName AS 'Nome da Loja',
	EmployeeCount AS 'Quantidade Colaboradores'
FROM DimStore
ORDER BY EmployeeCount DESC
"""
pd.read_sql(consulta_7, cnn)
#Com essa consulta temos que a loja com mais funcionário é a 'Contoso North America Online Store' e possui 325 funcionários.

,Nome da Loja,Quantidade Colaboradores
0,Contoso North America Online Store,325.0
1,Contoso Guangzhou Store,131.0
2,Contoso Catalog Store,120.0
3,Contoso Shanghai No.1 Store,95.0
4,Contoso Shanghai No.2 Store,95.0
...,...,...
301,Contoso Venezia Store,11.0
302,Contoso Strasbourg Store,10.0
303,Contoso Asia Online Store,8.0
304,Contoso Europe Online Store,7.0


/*b. Quantos funcionários tem a loja com menos funcionários?*/

In [ ]:
consulta_8 = """
SELECT
	StoreName AS 'Nome da Loja',
	EmployeeCount AS 'Quantidade Colaboradores'
FROM DimStore
ORDER BY EmployeeCount ASC
"""
pd.read_sql(consulta_8, cnn)
#Com essa consulta temos que a loja com menor número de funcionários é a 'Contoso Europe Online Store' e possui somente 7 funcionários.

,Nome da Loja,Quantidade Colaboradores
0,Contoso Cheshire Store,NaN
1,Contoso Europe Online Store,7.0
2,Contoso Asia Online Store,8.0
3,Contoso Strasbourg Store,10.0
4,Contoso Manchester Store,11.0
...,...,...
301,Contoso Saint Petersburg Store,95.0
302,Contoso Nizhny Novgorod Store,95.0
303,Contoso Catalog Store,120.0
304,Contoso Guangzhou Store,131.0


/*A área de RH está com uma nova ação para a empresa, e para isso precisa
saber a quantidade total de funcionários do sexo Masculino e do sexo
Feminino.

a. Descubra essas duas informações utilizando o SQL*/

In [ ]:
consulta_9 = """
SELECT
	COUNT(Gender) AS 'Quantidade M'
FROM DimEmployee
WHERE Gender = 'M'
"""
pd.read_sql(consulta_9, cnn)
#Com essa consulta temos que a quantidade de funcionários do sexo 'Masculino' é de 206.

,Quantidade M
0,206


In [ ]:
consulta_10 = """

SELECT
	COUNT(Gender) AS 'Quantidade F'
FROM DimEmployee
WHERE Gender = 'F'
"""
pd.read_sql(consulta_10, cnn)
#Com essa consulta temos que a quantidade de funcionários do sexo 'Masculino' é de 87.

,Quantidade F
0,87


/*O funcionário e a funcionária mais antigos receberão uma
homenagem. Descubra as seguintes informações de cada um deles:
Nome, E-mail, Data de Contratação.*/

In [ ]:
#O HOMEM
consulta_11 = """
SELECT
	FirstName AS 'Nome Funcionario',
	LastName AS 'Sobrenome Funcionario',
	EmailAddress AS 'E-mail',
	StartDate AS 'Data Contratacao',
	Gender AS 'Sexo'
FROM DimEmployee
WHERE Gender = 'M'
ORDER BY StartDate ASC
LIMIT 1
"""

pd.read_sql(consulta_11, cnn)
#Com essa consulta temos que o funcionário mais antigo, do sexo masculino é o  'Kim Abercrombie' e sua data de nascimento é de 31/07/1996.

,Nome Funcionario,Sobrenome Funcionario,E-mail,Data Contratacao,Sexo
0,Kim,Abercrombie,guy1@contoso.com,1996-07-31,M


In [ ]:
#A MULHER
consulta_12 = """
SELECT
	FirstName AS 'Nome Funcionario',
	LastName AS 'Sobrenome Funcionario',
	EmailAddress AS 'E-mail',
	StartDate AS 'Data Contratacao',
	Gender AS 'Sexo'
FROM DimEmployee
WHERE Gender = 'F'
ORDER BY StartDate ASC
LIMIT 1
"""

pd.read_sql(consulta_12, cnn)

#Com essa consulta temos que a funcionária mais antiga, do sexo feminino é a 'Terry Adams' e sua data de nascimento é de 26/01/1998.

,Nome Funcionario,Sobrenome Funcionario,E-mail,Data Contratacao,Sexo
0,Terry,Adams,jolynn0@contoso.com,1998-01-26,F


/*Agora você precisa fazer uma análise dos produtos. Será necessário
descobrir as seguintes informações:
a. Quantidade distinta de cores de produtos.
b. Quantidade distinta de marcas
c. Quantidade distinta de classes de produto
Para simplificar, você pode fazer isso em uma mesma consulta.*/

In [ ]:
consulta_13 = """
SELECT
	COUNT(DISTINCT ColorName) AS 'Qtd de Cores Distintas',
	COUNT(DISTINCT BrandName) AS 'Qtd de Marcas Distintas',
	COUNT(DISTINCT ClassName) AS 'Qtd de Classes Distintas'
FROM DimProduct
"""
pd.read_sql(consulta_13, cnn)
#Com essa consulta temos que a quantidade de 'Cores Distintas',Marcas Distintas',Classes Distintas' são respectivamente 17,15,3.

,Qtd de Cores Distintas,Qtd de Marcas Distintas,Qtd de Classes Distintas
0,17,15,3


###Analisando Dados do BD Contoso com SQL (Where/Like)


/*Você é responsável pelo setor de logística da empresa Contoso e precisa
dimensionar o transporte de todos os produtos em categorias, de acordo
com o peso.
Os produtos da categoria A, com peso acima de 100kg, deverão ser
transportados na primeira leva.
Faça uma consulta no banco de dados para descobrir quais são estes
produtos que estão na categoria A.
a. Você deverá retornar apenas 2 colunas nessa consulta: Nome do
Produto e Peso.*/

In [ ]:
consulta_14 = """
SELECT
	ProductName AS 'Nome Produto',
	Weight AS 'Peso'
FROM DimProduct
WHERE Weight > 100
ORDER BY Weight ASC

"""
pd.read_sql(consulta_14, cnn)
#Com essa consulta temos que, há 112 produtos com peso acima de 100, variando de 102 á 239.

,Nome Produto,Peso
0,Litware Washer & Dryer 21in E214 Blue,102
1,NT Washer & Dryer 27in L2700 Blue,102
2,Contoso Washer & Dryer 27in L270 Silver,102
3,Litware Washer & Dryer 15.5in E150 Blue,105
4,NT Washer & Dryer 25.5in M2550 Blue,105
...,...,...
1936,Contoso Air conditioner 25000BTU L1672 White,96
1937,Contoso Air conditioner 8000BTU M0320 Blue,96
1938,Proseware Air conditioner 25000BTU L167 Grey,96
1939,Contoso Air conditioner 12000BTU M0640 Blue,98


/*Você foi alocado para criar um relatório das lojas registradas atualmente
na Contoso.
a. Descubra quantas lojas a empresa tem no total. Na consulta que
você deverá fazer à tabela DimStore, retorne as seguintes
informações: StoreName, OpenDate, EmployeeCount.*/

In [ ]:
consulta_15 = """
SELECT 
	StoreName AS 'Nome da Loja',
	OpenDate AS 'Data de Abertura',
	EmployeeCount AS 'Numero de Colaboradores'
FROM DimStore
WHERE StoreType = 'Store' AND Status = 'On'
ORDER BY EmployeeCount DESC
"""
pd.read_sql(consulta_15, cnn)
#Com essa consulta temos que a loja com maior número de colaboradores é a 'Contoso Guangzhou Store' com 131 colaboradores.

,Nome da Loja,Data de Abertura,Numero de Colaboradores
0,Contoso Guangzhou Store,2009-07-17 00:00:00.000,131.0
1,Contoso Shanghai No.1 Store,2004-02-14 00:00:00.000,95.0
2,Contoso Shanghai No.2 Store,2004-03-02 00:00:00.000,95.0
3,Contoso Ottawa No.1 Store,2004-04-09 00:00:00.000,95.0
4,Contoso Ottawa No.2 Store,2004-05-15 00:00:00.000,95.0
...,...,...,...
282,Contoso Manchester Store,2004-06-19 00:00:00.000,11.0
283,Contoso Nantes Store,2004-08-03 00:00:00.000,11.0
284,Contoso Venezia Store,2009-07-06 00:00:00.000,11.0
285,Contoso Strasbourg Store,2004-09-04 00:00:00.000,10.0


 /*O gerente da área de controle de qualidade notificou à Contoso que
todos os produtos Home Theater da marca Litware, disponibilizados
para venda no dia 15 de março de 2009, foram identificados com
defeitos de fábrica.
O que você deverá fazer é identificar os ID’s desses produtos e repassar
ao gerente para que ele possa notificar as lojas e consequentemente
solicitar a suspensão das vendas desses produtos.*/

In [ ]:
consulta_16 = """
SELECT
	ProductKey AS 'ID',
	ProductName AS 'Nome do Produto',
	BrandName AS 'Marca',
	AvailableForSaleDate AS 'Data Disponivel Venda'
FROM DimProduct
WHERE ProductName LIKE '%Home Theater%' AND BrandName = 'Litware' AND AvailableForSaleDate  LIKE '%2009-03-15%'
"""
pd.read_sql(consulta_16, cnn)
#Com essa consulta temos que há o retorno de 3 produtos. Nessa consulta usamos a clausula WHERE com 3 condições (ProductName,BrandName,AvailableForSaleDate).

,ID,Nome do Produto,Marca,Data Disponivel Venda
0,198,Litware Home Theater System 4.1 Channel M411 B...,Litware,2009-03-15 00:00:00.000
1,214,Litware Home Theater System 4.1 Channel M411 S...,Litware,2009-03-15 00:00:00.000
2,230,Litware Home Theater System 4.1 Channel M411 B...,Litware,2009-03-15 00:00:00.000


/*Imagine que você precise extrair um relatório da tabela DimStore, com
informações de lojas. Mas você precisa apenas das lojas que não estão
mais funcionando atualmente.
a. Utilize a coluna de Status para filtrar a tabela e trazer apenas as lojas
que não estão mais funcionando.*/

In [ ]:
consulta_17 = """
SELECT 
	StoreName AS 'Nome da Loja',
	Status AS 'Situação'
FROM DimStore
WHERE Status <> 'On'
"""
pd.read_sql(consulta_17, cnn)
#Com essa consulta temos que há o retorno de 11 Lojas com a Situação 'Off'. Foi utilizado o operador lógico '<>' (diferente).

,Nome da Loja,Situação
0,Contoso North Bend Store,Off
1,Contoso Cheney Store,Off
2,Contoso Sedalia Store,Off
3,Contoso Milwaukee No.2 Store,Off
4,Contoso Stoughton Store,Off
5,Contoso Humble Store,Off
6,Contoso Key West Store,Off
7,Contoso Buffalo Store,Off
8,Contoso Trenton No.2 Store,Off
9,Contoso Worcester No.2 Store,Off


/*De acordo com a quantidade de funcionários, cada loja receberá uma
determinada quantidade de máquinas de café. As lojas serão divididas em
3 categorias:
CATEGORIA 1: De 1 a 20 funcionários -> 1 máquina de café
CATEGORIA 2: De 21 a 50 funcionários -> 2 máquinas de café
CATEGORIA 3: Acima de 51 funcionários -> 3 máquinas de café
Identifique, para cada caso, quais são as lojas de cada uma das 3
categorias acima (basta fazer uma verificação).*/

In [ ]:
#Categoria 1
consulta_18 = """
SELECT
	StoreName AS 'Nome Loja',
	EmployeeCount AS 'Qtd Funcionario'
FROM DimStore
WHERE EmployeeCount <= 20
ORDER BY EmployeeCount DESC
"""
pd.read_sql(consulta_18, cnn)
#Com essa consulta temos o resultado das lojas que irão receber ' 1 máquina de café '

,Nome Loja,Qtd Funcionario
0,Contoso Sydney No.1 Store,20.0
1,Contoso Bellevue Store,19.0
2,Contoso North Bend Store,19.0
3,Contoso Cheney Store,19.0
4,Contoso Lafayette Store,19.0
...,...,...
70,Contoso Nantes Store,11.0
71,Contoso Venezia Store,11.0
72,Contoso Strasbourg Store,10.0
73,Contoso Asia Online Store,8.0


In [ ]:
#Categoria 2
consulta_19 = """
SELECT
	StoreName AS 'Nome Loja',
	EmployeeCount AS 'Qtd Funcionario'
FROM DimStore
WHERE EmployeeCount > 20 AND EmployeeCount <=50
ORDER BY EmployeeCount DESC
"""
pd.read_sql(consulta_19, cnn)
#Com essa consulta temos o resultado das lojas que irão receber ' 2 máquinas de café '

,Nome Loja,Qtd Funcionario
0,Contoso Nagoya Store,50.0
1,Contoso Sapporo Store,50.0
2,Contoso Kyoto Store,50.0
3,Contoso Seoul Store,50.0
4,Contoso Busan Store,50.0
...,...,...
182,Contoso Kennebunkport Store,22.0
183,Contoso Darien Store,22.0
184,Contoso Alexandria Store,22.0
185,Contoso Annapolis Store,22.0


In [ ]:
#Categoria 3
consulta_20 = """
SELECT
	StoreName AS 'Nome Loja',
	EmployeeCount AS 'Qtd Funcionario'
FROM DimStore
WHERE EmployeeCount > 51
ORDER BY EmployeeCount DESC
"""
pd.read_sql(consulta_20, cnn)
#Com essa consulta temos o resultado das lojas que irão receber ' 3 máquina de café '

,Nome Loja,Qtd Funcionario
0,Contoso North America Online Store,325.0
1,Contoso Guangzhou Store,131.0
2,Contoso Catalog Store,120.0
3,Contoso Shanghai No.1 Store,95.0
4,Contoso Shanghai No.2 Store,95.0
5,Contoso Ottawa No.1 Store,95.0
6,Contoso Ottawa No.2 Store,95.0
7,Contoso Toronto No.1 Store,95.0
8,Contoso Toronto No.2 Store,95.0
9,Contoso Toronto No.3 Store,95.0


/*A empresa decidiu que todos os produtos LCD receberão um super
desconto no próximo mês. O seu trabalho é fazer uma consulta à tabela
DimProduct e retornar os ID’s, Nomes e Preços de todos os produtos
LCD existentes.*/

In [ ]:
consulta_21 = """
SELECT
	ProductKey AS 'ID',
	ProductName AS 'Nome Produto',
	UnitPrice AS 'Preço Produto'
FROM DimProduct
WHERE ProductName LIKE '%LCD%'
"""
pd.read_sql(consulta_21, cnn)
#Com essa consulta temos o resultado de todos os produtos 'LCD', foi utilizado a clausula "LIKE", 
#que tem como função retornar tudo aquilo que contem 'LCD' da coluna 'ProductName'.

,ID,Nome Produto,Preço Produto
0,122,"Adventure Works 19"" Portable LCD HDTV M110 Silver","279,99"
1,123,"Adventure Works 19"" Portable LCD HDTV M110 Black","279,99"
2,124,"Adventure Works 19"" Portable LCD HDTV M110 White","279,99"
3,133,"Adventure Works 20"" LCD HDTV M120 Silver","349,95"
4,134,"Adventure Works 20"" LCD HDTV M120 Black","349,95"
...,...,...,...
139,637,WWI Projector 480p LCD12 Silver,"229,00"
140,1424,The Phone Company Touch Screen Phones - LCD M1...,"200,00"
141,1441,The Phone Company Touch Screen Phones - LCD M1...,"200,00"
142,1458,The Phone Company Touch Screen Phones - LCD M1...,"200,00"


/*Faça uma lista com todos os produtos das cores: Green, Orange, Black,
Silver e Pink. Estes produtos devem ser exclusivamente das marcas:
Contoso, Litware e Fabrikam.*/

In [ ]:
consulta_22 = """
SELECT
	ProductName AS 'Nome Produto',
	ColorName AS 'Cor',
	BrandName AS 'Marca'

FROM DimProduct
WHERE ColorName  IN (
						'Green','Orange','Black', 'Silver','Pink'
						) AND 
		BrandName IN (
						'Contoso','Litware','Fabrikan'
						)
"""
pd.read_sql(consulta_22, cnn)
#Com essa consulta temos o resultado dos produtos das marcas Contoso, Litware e Fabrikam que possuem cores Green, Orange, Black, Silver e Pink.
#Para isso foi utilizado os operadores lógicos IN e AND, onde o primeiro considera dentro do intervalo se atende um ou outra condição, já o operador AND 
#só retorna uma determinada consulta se as duas ou mais condições forem atendidas.

,Nome Produto,Cor,Marca
0,Contoso 512MB MP3 Player E51 Silver,Silver,Contoso
1,Contoso 2G MP3 Player E200 Silver,Silver,Contoso
2,Contoso 2G MP3 Player E200 Black,Black,Contoso
3,Contoso 4G MP3 Player E400 Silver,Silver,Contoso
4,Contoso 4G MP3 Player E400 Black,Black,Contoso
...,...,...,...
403,Contoso Original K1m Li-Ion Standard Battery E...,Black,Contoso
404,Contoso Original K1m Li-Ion Standard Battery E...,Silver,Contoso
405,Contoso Bluetooth Active Headphones L15 Black,Black,Contoso
406,Contoso In-Line Coupler E180 Black,Black,Contoso


######Analisando Dados do BD Contoso com SQL (Joins)

/*Faça um resumo da quantidade vendida (Sales Quantity) de acordo
com o nome do canal de vendas (ChannelName). Você deve ordenar a
tabela final de acordo com SalesQuantity, em ordem decrescente.*/

In [ ]:
consulta_24 = """
SELECT 
	ChannelName AS 'Nome Canal Vendas',
	SUM(SalesQuantity) AS 'Quantidade Vendida'
FROM
	FactSales
LEFT JOIN DimChannel
	ON FactSales.channelKey = DimChannel.ChannelKey
GROUP BY
	ChannelName
ORDER BY
	[Quantidade Vendida] DESC
"""
pd.read_sql(consulta_24, cnn)
#Com essa consulta temos o resumo dos canais de vendas e suas respectivas 'Quantidade de Vendas' , ordenadas por essa coluna.

,Nome Canal Vendas,Quantidade Vendida
0,Store,83692
1,Online,38965
2,Reseller,21310
3,Catalog,16768


/*Faça um resumo do valor total vendido (Sales Amount) para cada mês
(CalendarMonthLabel) e ano (CalendarYear).*/


In [ ]:
consulta_26 = """
SELECT
	
	CalendarYear 'Ano',
	CalendarMonthLabel 'Mes',
	SUM(SalesAmount) AS 'Faturamento'
FROM
	FactSales
LEFT JOIN DimDate
	ON FactSales.DateKey = DimDate.Datekey
GROUP BY
	CalendarYear,
	CalendarMonthLabel,
	CalendarMonth
ORDER BY
	CalendarMonth
"""
pd.read_sql(consulta_26, cnn)
#Com essa consulta temos o resultado do 'Faturamento' de cada ano e de cada mes de cada ano, pra exibir o resultado é necessário colocar todas as colunas que
#irão aparecer no SELECT , na clausula Group By, só não a agregação.

,Ano,Mes,Faturamento
0,2007,January,942231.0
1,2007,February,821525.0
2,2007,March,795564.0
3,2007,April,1303146.0
4,2007,May,1296768.0
5,2007,June,1252984.0
6,2007,July,1229334.0
7,2007,August,1043516.0
8,2007,September,1151824.0
9,2007,October,1215005.0


/*Você precisa fazer uma análise de vendas por produtos. O objetivo final é
descobrir o valor total vendido (SalesAmount) por produto.
a. Descubra qual é a cor de produto que mais é vendida (de acordo com
SalesQuantity).*/

In [ ]:
consulta_27 = """
SELECT 
	ColorName AS 'Cor',
	SUM(SalesQuantity) AS 'Total Quantidade Vendida'
	
FROM
	FactSales
LEFT JOIN DimProduct
	ON FactSales.ProductKey = DimProduct.ProductKey
GROUP BY
	ColorName
ORDER BY 
	SUM(SalesQuantity) DESC
LIMIT 1
"""
pd.read_sql(consulta_27, cnn)
#Com essa consulta temos o resultado que a cor que mais vende é a Black.

,Cor,Total Quantidade Vendida
0,Black,39892


/*b. Quantas cores tiveram uma quantidade vendida acima de 30.000.*/

In [ ]:
consulta_28 = """
SELECT 
	ColorName AS 'Cor',
	SUM(SalesQuantity) AS 'Total Quantidade Vendida'
	
FROM
	FactSales
LEFT JOIN DimProduct
	ON FactSales.ProductKey = DimProduct.ProductKey
GROUP BY
	ColorName
HAVING
	SUM(SalesQuantity) > 30000
ORDER BY 
	SUM(SalesQuantity) DESC
"""
pd.read_sql(consulta_28, cnn)
#Com essa consulta temos o resultado que as cores que venderam masi de 30.000 unidades, são Black, Silver, White.

,Cor,Total Quantidade Vendida
0,Black,39892
1,Silver,33169
2,White,32030


/*Crie um agrupamento de quantidade vendida (SalesQuantity) por
categoria do produto (ProductCategoryName).


In [ ]:
consulta_29 = """
SELECT
	ProductCategoryName AS 'Nome Categoria',
	SUM(SalesQuantity) AS 'Total Quantidade Vendida'
FROM
	FactSales
INNER JOIN DimProduct
	ON FactSales.ProductKey = DimProduct.ProductKey
		INNER JOIN DimProductSubcategory
		ON DimProduct.ProductSubcategoryKey = DimProductSubcategory.ProductSubcategoryKey
			INNER JOIN DimProductCategory
				ON DimProductCategory.ProductCategoryKey =  DimProductSubcategory.ProductCategoryKey
GROUP BY
	ProductCategoryName
ORDER BY
	SUM(SalesQuantity) DESC
"""
pd.read_sql(consulta_29, cnn)
#Com essa consulta temos o resultado das categorias e os totais vendidos. Foi necessário utilizar mais de um Join, visto que as tabelas não se relacionvam de 
#forma direta com a 'FactSales'.

,Nome Categoria,Total Quantidade Vendida
0,Cell phones,42873
1,Computers,32225
2,Home Appliances,30634
3,Cameras and camcorders,18955
4,Games and Toys,17413
5,TV and Video,9776
6,"Music, Movies and Audio Books",4485
7,Audio,4374


###Analisando Dados do BD Contoso com SQL (subConsulta)

/* Retornar uma tabela com todos os produtos (ID Produto e Nome Produto) que possuem alguma venda no dia 02/01/2007

*/

In [ ]:
consulta_30 = """
SELECT
	ProductKey,
	ProductName
FROM
		DimProduct
WHERE EXISTS (
	SELECT	
		ProductKey
	FROM
		FactSales
	WHERE
		DateKey LIKE '2007-01-02%'
		AND FactSales.ProductKey = DimProduct.ProductKey 
		)


"""
pd.read_sql(consulta_30, cnn)

,ProductKey,ProductName
0,460,WWI Desktop PC1.80 E1802 White
1,610,Contoso Screen 80in E080 Silver
2,956,A. Datum Point Shoot Digital Camera M500 Black
3,1045,"A. Datum SLR Camera 35"" X358 Black"
4,1658,Contoso DVD 7-Inch Player Portable E200 White
5,2036,Litware Microwave 0.8CuFt E080 Grey
6,2059,Contoso Microwave 0.9CuFt E0090 White
7,2145,Adventure Works Coffee Maker 4C E080 Black
